In [13]:
import pandas as pd
import numpy as np
import math
import scipy.stats as st
import networkx as nx
import matplotlib.pyplot as plt

In [14]:
# Load dataset
data = pd.read_csv('titanic_data.csv')
COUNT = data.count()[0]
print(data.describe())
print('- - - - - - - - - - - - - - - - - - - -')

         Survived      Pclass  ...  Parents/Children Aboard       Fare
count  887.000000  887.000000  ...               887.000000  887.00000
mean     0.385569    2.305524  ...                 0.383315   32.30542
std      0.487004    0.836662  ...                 0.807466   49.78204
min      0.000000    1.000000  ...                 0.000000    0.00000
25%      0.000000    2.000000  ...                 0.000000    7.92500
50%      0.000000    3.000000  ...                 0.000000   14.45420
75%      1.000000    3.000000  ...                 0.000000   31.13750
max      1.000000    3.000000  ...                 6.000000  512.32920

[8 rows x 7 columns]
- - - - - - - - - - - - - - - - - - - -


In [15]:
# Variable transform
CATEGORICAL_FEATURES = ['Survived', 'Pclass', 'Sex', 'Siblings/Spouses Aboard', 'Parents/Children Aboard']
NUMERICAL_FEATURES = ['Age', 'Fare']
ALL_FEATURES = CATEGORICAL_FEATURES + NUMERICAL_FEATURES
for feature_name in CATEGORICAL_FEATURES:
    data = data.join(pd.get_dummies(data[feature_name], prefix=feature_name))
    data.drop([feature_name], axis = 1, inplace=True)
for feature_name in NUMERICAL_FEATURES:
    data = data.join(pd.get_dummies(pd.qcut(data[feature_name], 5), prefix=feature_name))
    data.drop([feature_name], axis = 1, inplace=True)
features = list(data.columns)
print('First, I divided these six features into two categories.')
print('CATEGORICAL_FEATURES and NUMERICAL_FEATURES')
print('For CATEGORICAL_FEATURES, I directly convert it to one-hot encoding.')
print('For NUMERICAL_FEATURES, binning the data and then convert it to one-hot encoding.')
print(data.describe())
print(data)
print('- - - - - - - - - - - - - - - - - - - -')

First, I divided these six features into two categories.
CATEGORICAL_FEATURES and NUMERICAL_FEATURES
For CATEGORICAL_FEATURES, I directly convert it to one-hot encoding.
For NUMERICAL_FEATURES, binning the data and then convert it to one-hot encoding.
       Survived_0  Survived_1  ...  Fare_(22.225, 39.688]  Fare_(39.688, 512.329]
count  887.000000  887.000000  ...             887.000000              887.000000
mean     0.614431    0.385569  ...               0.201804                0.198422
std      0.487004    0.487004  ...               0.401573                0.399036
min      0.000000    0.000000  ...               0.000000                0.000000
25%      0.000000    0.000000  ...               0.000000                0.000000
50%      1.000000    0.000000  ...               0.000000                0.000000
75%      1.000000    1.000000  ...               0.000000                0.000000
max      1.000000    1.000000  ...               1.000000                1.000000

[8 rows x

In [16]:
# Entropy
def Entropy(FEATURE, data):
    ans = 0
    for col in features:
        if col[:len(FEATURE)] == FEATURE:
            pr = data[col].sum()/data[col].count()
            if pr == 0:
                continue
            ans = ans - pr * math.log(pr)
    return ans

# Conditional Entropy
def H(FEATURE1, FEATURE2, data):
    ans = 0
    for col_y in features:
        if col_y[:len(FEATURE2)] == FEATURE2:
            Pr_val = data[col_y].sum() / data[col_y].count()
            sub_ans = 0
            for col_x in features:
                if col_x[:len(FEATURE1)] == FEATURE1:
                    pr = np.sum(data[col_x]+data[col_y] == 2) / data[col_y].sum()
                    if pr == 0:
                        continue
                    sub_ans = sub_ans - pr * math.log(pr)
            ans = ans + Pr_val * sub_ans
    return ans

# Mutual Information
def I(FEATURE1, FEATURE2, data):
    return Entropy(FEATURE1, data) - H(FEATURE1, FEATURE2, data)

In [17]:
H(CATEGORICAL_FEATURES[1], CATEGORICAL_FEATURES[0], data)

0.9420283503592444

In [22]:
# Find Max_Mutual_Information_feature
def Find_Max_Mutual_Information_feature(DATA, ALL_FEATURES):
    y_label = ALL_FEATURES[0]
    Max_Mutual_Information_feature_name = ALL_FEATURES[1]
    Max_Mutual_Information = I(Max_Mutual_Information_feature_name, y_label, DATA)
    for feature_name in ALL_FEATURES[1:]:
        MI = I(feature_name, y_label, DATA)
        if MI > Max_Mutual_Information:
            Max_Mutual_Information_feature_name = feature_name
            Max_Mutual_Information = MI
    print(Max_Mutual_Information)
    return Max_Mutual_Information_feature_name

In [23]:
# Split Data
Selected_Feature = Find_Max_Mutual_Information_feature(data, ALL_FEATURES)
New_ALL_FEATURES = [Feature for Feature in ALL_FEATURES if Feature != Selected_Feature]
features = list(data.columns)
vals = [val for val in features if val[:len(Selected_Feature)] == Selected_Feature]
for val in vals:
    New_data = data[data[val] == 1].drop(vals, axis=1)
    entropy = Entropy(ALL_FEATURES[0], New_data)
    New_Selected_Feature = Find_Max_Mutual_Information_feature(New_data, New_ALL_FEATURES)
    print(New_Selected_Feature, entropy, val, val[len(Selected_Feature)+1:])
print(features[0], features[1])

0.15030862287961178
0.026185133916817538
Fare 0.4865517701249331 Sex_0 0
0.1438780085605199
Pclass 0.5709141922481396 Sex_1 1
Survived_0 Survived_1


In [20]:
class Node:
    def __init__(self, feature, value = None):
        self.feature = feature
        self.val = value
        self.children = []
        self.children_num = 0

    def leaf(self, val, feature):
        LEAF = Node(feature, val)
        self.children.append(LEAF)
        self.children_num = self.children_num + 1

In [21]:
def BuildTree(data, ALL_FEATURES, val = None, epsilon = 1e-3, alpha = 0.05):
    Selected_Feature = Find_Max_Mutual_Information_feature(data, ALL_FEATURES)
    node = Node(Selected_Feature, val)
    features = list(data.columns)
    New_ALL_FEATURES = [Feature for Feature in ALL_FEATURES if Feature != Selected_Feature]
    vals = [val for val in features if val[:len(Selected_Feature)] == Selected_Feature]
    for val in vals:
        New_data = data[data[val] == 1].drop(vals, axis=1)

        # stopping criteria
        entropy = Entropy(ALL_FEATURES[0], New_data)
        if entropy < epsilon or New_data.count()[0] < alpha * COUNT or len(New_ALL_FEATURES) < 2:
            if New_data[features[0]].sum() > New_data[features[1]].sum():
                node.leaf(val[len(Selected_Feature)+1:], features[0])
            else:
                node.leaf(val[len(Selected_Feature)+1:], features[1])
            return node

        ####################
        node.children.append(BuildTree(New_data, New_ALL_FEATURES, val[len(Selected_Feature)+1:]))
        node.children_num = node.children_num + 1
    return node
DecisionTree = BuildTree(data, ALL_FEATURES)

Sex
Fare
Age
Age
Parents/Children Aboard
Siblings/Spouses Aboard
Age
Pclass
Age
Siblings/Spouses Aboard
Pclass
Parents/Children Aboard
Fare
Age
Fare


<ipython-input-16-68f10846e613>:6: RuntimeWarning: invalid value encountered in longlong_scalars
  pr = data[col].sum()/data[col].count()
<ipython-input-16-68f10846e613>:6: RuntimeWarning: invalid value encountered in longlong_scalars
  pr = data[col].sum()/data[col].count()


In [10]:
# Display the Tree

In [11]:
# Predict
def Predict(x, DecisionTree):
    return x




print(Predict(x, DecisionTree))

NameError: name 'x' is not defined